# 🌴 Other Crops Processing
---

This notebook processes the data found in [link](https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2E__CS/0062E4EVCP1.px/?rxid=bdf9d8da-96f1-4100-ae09-18cb3eaeb313). 

The data was downloaded in batches of csv files.

## Methodology Notes
---

- We define a dictionary `region_province` and `prov_to_reg` that maps the relationship between provinces and regions.

- We include a categorical indicator for broad crop types: `Non-Food and Industrial Crops`, `Fruit Crops`, and `Vegetables and Root Crops` as per the [Crops Production Survey](https://psada.psa.gov.ph/index.php/catalog/254).

- We create a hierarchical time series dataset where the lowest level is defined by Crop-Province.

- The raw dataset contains 343 "crops". However, some crops which can be further disaggregated are mixed with their aggregated versions (e.g. `Sugarcane`).

- See `crop-to-commodity.csv` for a list of aggregated/disaggregated crops and their commodity group.

- We manually add a `Coffee` crop that aggregates all the coffee.

- We merge with the palay and corn dataset (classified as `Cereals` in terms of commodity group) downstream.

- TODO: Create a hierarchical diagram/table similar to M5.

## Notes on Crops
---

- Some of the names in the raw dataset are misspelled, meaning they don't match the terms used in official Manual of Operations.

- The `Dracaena`, `D.Marginata Tricolor`, `D.Sanderiana White (Whitecorn)`, `D.Sanderiana Yellow`, `Song of Korea`, and `Florida Beauty` time series don't seem to aggregate together.

- The `Cassava` time series was only disaggregated into `Cassava for Industrial use` and `Cassava for food` after 2015.

- Strangely, `Jackfruit young` is classified as `Vegetables and Root Crops`, while `Jackfruit, ripe` is classified as `Fruit Crops`.

- `Loquat` has no recorded production, ever.

- `Coffee` (which is not in the raw dataset) is split into `Coffee (dried berries w/ pulp)` and `Coffee (green coffee beans)`.

- There may be some bugs in the `crop-to-commodity.csv` file.

## Footnotes
---

- Preliminary data for Quarter 4, Semester 2, and Annual of 2022

- BARMM excludes eight area clusters

- Starting the Third Quarter of 2023, discontinued production of coffee in dried berries form

- Revised: 2020 - Carrots (South Cotabato, Sultan Kudarat), Snapbeans (Batanes), Eggplant (South Cotabato, Sultan Kudarat), Greater Yam/Ubi (Davao Region), Pepper Finger (Iloilo, Negros Occidental), Banana Saba (Davao Occidental), Banana Others (Davao de Sur, Davao Occidental), Lanzones (Davao Occidental), Papaya Hawaiian (Negros Occidental), Papaya Native (Davao Region), Guyabano (All regions), Sineguelas (All Regions), Batawan (Agusan del Norte), Grapes Red (Negros Oriental), Makopa (Davao Occidental)

In [1]:
import glob
import numpy as np
import pandas as pd

region_province = {
    'CORDILLERA ADMINISTRATIVE REGION (CAR)': ["Abra", "Apayao", "Benguet", "Ifugao", "Kalinga", "Mountain Province"],
    'REGION I (ILOCOS REGION)': ["Ilocos Norte", "Ilocos Sur", "La Union", "Pangasinan"],
    'REGION II (CAGAYAN VALLEY)': ["Batanes", "Cagayan", "Isabela", "Nueva Vizcaya", "Quirino"],
    'REGION III (CENTRAL LUZON)': ["Aurora", "Bataan", "Bulacan", "Nueva Ecija", "Pampanga", "Tarlac", "Zambales"],
    'REGION IV-A (CALABARZON)': ["Batangas", "Cavite", "Laguna", "Quezon", "Rizal"],
    'MIMAROPA REGION': ["Marinduque", "Occidental Mindoro", "Oriental Mindoro", "Palawan", "Romblon"],
    'REGION V (BICOL REGION)': ["Albay", "Camarines Norte", "Camarines Sur", "Catanduanes", "Masbate", "Sorsogon"],
    'REGION VI (WESTERN VISAYAS)': ["Aklan", "Antique", "Capiz", "Guimaras", "Iloilo","Negros Occidental"],
    'REGION VII (CENTRAL VISAYAS)': ["Bohol", "Cebu", "Negros Oriental", "Siquijor"],
    'REGION VIII (EASTERN VISAYAS)': ["Biliran", "Eastern Samar", "Leyte", "Northern Samar", "Samar", "Southern Leyte"],
    'REGION IX (ZAMBOANGA PENINSULA)': ["Zamboanga del Norte", "Zamboanga del Sur", "Zamboanga Sibugay", "City of Zamboanga"],
    'REGION X (NORTHERN MINDANAO)': ["Bukidnon", "Camiguin", "Lanao del Norte", "Misamis Oriental", "Misamis Occidental"],
    'REGION XI (DAVAO REGION)': ["Davao de Oro", "Davao del Norte","Davao del Sur", "Davao Occidental", "Davao Oriental", "City of Davao"],
    'REGION XII (SOCCSKSARGEN)': ["Cotabato", "Sarangani", "Sultan Kudarat", "South Cotabato"],
    'REGION XIII (CARAGA)': ["Agusan del Norte", "Agusan del Sur", "Dinagat Islands", "Surigao del Norte", "Surigao del Sur"],
    'BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANAO (BARMM)': ["Basilan", "Lanao del Sur", "Maguindanao", "Sulu", "Tawi-tawi"]
 }

prov_to_reg = {}
for r, p_list in region_province.items():
    for p in p_list:
        prov_to_reg[p] = r

In [2]:
files = glob.glob('../raw/2E4EVCP1*.csv')
files

['../raw\\2E4EVCP1 (0).csv',
 '../raw\\2E4EVCP1 (1).csv',
 '../raw\\2E4EVCP1 (10).csv',
 '../raw\\2E4EVCP1 (11).csv',
 '../raw\\2E4EVCP1 (12).csv',
 '../raw\\2E4EVCP1 (13).csv',
 '../raw\\2E4EVCP1 (14).csv',
 '../raw\\2E4EVCP1 (15).csv',
 '../raw\\2E4EVCP1 (16).csv',
 '../raw\\2E4EVCP1 (17).csv',
 '../raw\\2E4EVCP1 (18).csv',
 '../raw\\2E4EVCP1 (2).csv',
 '../raw\\2E4EVCP1 (3).csv',
 '../raw\\2E4EVCP1 (4).csv',
 '../raw\\2E4EVCP1 (5).csv',
 '../raw\\2E4EVCP1 (6).csv',
 '../raw\\2E4EVCP1 (7).csv',
 '../raw\\2E4EVCP1 (8).csv',
 '../raw\\2E4EVCP1 (9).csv']

In [3]:
df = pd.concat([pd.read_csv(file, skiprows=2) for file in files], axis=0)
df['Level'] = df['Geolocation'].str[:4].str.replace('....', 'P', regex=False).str[:2].str.replace('..','R', regex=False)
df['Geolocation'] = df['Geolocation'].str.replace('^\.*', '', regex=True)
df = df[sorted(df.columns, reverse=True)]
df = df.set_index(['Crop', 'Geolocation', 'Level'])
df = df.sort_values(by=['Level', 'Crop', 'Geolocation'])
df = df.reset_index()

In [4]:
df_p = df[df['Level'] == 'P'].copy()
df_p = df_p.drop(['Level'], axis=1)
df_p = df_p.rename({'Geolocation': 'Province'}, axis=1)
df_p['Region'] = df_p['Province'].map(prov_to_reg)
df_p = df_p[sorted(df_p.columns, reverse=False)]
df_p = df_p.sort_values(by=['Crop', 'Province', 'Region'])
df_p = pd.concat([df_p.iloc[:, -3:], df_p.iloc[:, :-3]], axis=1)
ts_col_names = [str(q) for q in pd.period_range(start='2010Q1', end='2022Q4', freq='Q')]
df_p.columns = ['Crop', 'Province', 'Region'] + ts_col_names
df_p[ts_col_names] = df_p[ts_col_names].replace('..', np.nan).astype(float)
df_p = df_p.fillna(0)
df_p = df_p.set_index(['Crop', 'Province', 'Region'])
df_p = df_p.T
df_p.index = pd.PeriodIndex(df_p.index, freq='Q')
df_p

Crop                    Abaca (dried raw fiber)                       \
Province                                   Abra     Agusan del Norte   
Region   CORDILLERA ADMINISTRATIVE REGION (CAR) REGION XIII (CARAGA)   
2010Q1                                      0.0               110.41   
2010Q2                                      0.0               135.16   
2010Q3                                      0.0               163.90   
2010Q4                                      0.0               138.61   
2011Q1                                      0.0                80.10   
2011Q2                                      0.0               117.76   
2011Q3                                      0.0               168.50   
2011Q4                                      0.0               142.00   
2012Q1                                      0.0                78.32   
2012Q2                                      0.0               110.03   
2012Q3                                      0.0               215.46   
2012Q4                                      0.0               125.36   
2013Q1                                      0.0                31.06   
2013Q2                                      0.0               131.00   
2013Q3                                      0.0               225.00   
2013Q4                                      0.0               134.18   
2014Q1                                      0.0                35.00   
2014Q2                                      0.0               144.10   
2014Q3                                      0.0               230.49   
2014Q4                                      0.0               137.45   
2015Q1                                      0.0                75.00   
2015Q2                                      0.0               145.00   
2015Q3                                      0.0               341.38   
2015Q4                                      0.0               164.00   
2016Q1                                      0.0               116.19   
2016Q2                                      0.0               155.00   
2016Q3                                      0.0               345.00   
2016Q4                                      0.0               177.00   
2017Q1                                      0.0               125.00   
2017Q2                                      0.0               162.64   
2017Q3                                      0.0               355.52   
2017Q4                                      0.0               195.00   
2018Q1                                      0.0               140.00   
2018Q2                                      0.0               171.13   
2018Q3                                      0.0               382.12   
2018Q4                                      0.0               207.88   
2019Q1                                      0.0               142.32   
2019Q2                                      0.0               175.00   
2019Q3                                      0.0               385.00   
2019Q4                                      0.0               285.00   
2020Q1                                      0.0               200.00   
2020Q2                                      0.0               210.00   
2020Q3                                      0.0               390.00   
2020Q4                                      0.0               390.00   
2021Q1                                      0.0               210.10   
2021Q2                                      0.0               224.00   
2021Q3                                      0.0               398.00   
2021Q4                                      0.0               387.35   
2022Q1                                      0.0               172.49   
2022Q2                                      0.0               170.00   
2022Q3                                      0.0               360.00   
2022Q4                                      0.0               390.83   

Crop                                   

In [5]:
crop_to_commodity = pd.read_csv('../data/crop-to-commodity.csv')
crop_to_commodity[['Aggregated', 'Disaggregated']] = crop_to_commodity[['Aggregated', 'Disaggregated']].fillna(False)
crop_to_commodity

,Crop-Raw,Crop,Crop-Type,Aggregated,Disaggregated
0,Abaca (dried raw fiber),Abaca,Non-Food and Industrial Crops,False,False
1,Abaca Leafsheath,NaN,Non-Food and Industrial Crops,False,False
2,Abiu,NaN,Fruit Crops,False,False
3,African Palm Leaves,NaN,Non-Food and Industrial Crops,False,False
4,Agitway,NaN,Vegetables and Root Crops,False,False
...,...,...,...,...,...
338,Yellow Bell,NaN,Non-Food and Industrial Crops,False,False
339,Yerba Buena,NaN,Non-Food and Industrial Crops,False,False
340,Young corn,Young Corn,Vegetables and Root Crops,False,False
341,Zapote,Sapote,Fruit Crops,False,False


In [6]:
df_augment = df_p.T.reset_index()
df_augment.insert(loc=1, column='Commodity', value=np.nan)
df_augment.insert(loc=2, column='Aggregated', value=np.nan)
df_augment.insert(loc=3, column='Disaggregated', value=np.nan)
df_augment = df_augment.astype({'Aggregated': 'bool', 'Disaggregated': 'bool'})

for idx, c in crop_to_commodity.iterrows():
    df_augment.loc[df_augment['Crop'] == c['Crop-Raw'], 'Commodity'] = c['Crop-Type']
    df_augment.loc[df_augment['Crop'] == c['Crop-Raw'], 'Aggregated'] = c['Aggregated']
    df_augment.loc[df_augment['Crop'] == c['Crop-Raw'], 'Disaggregated'] = c['Disaggregated']
    if c['Crop'] is not np.nan:
        df_augment.loc[df_augment['Crop'] == c['Crop-Raw'], 'Crop'] = c['Crop']

In [7]:
# Create coffee
df_coffee = df_augment[df_augment['Crop'].isin(['Coffee - Dried Berries', 'Coffee - Green Beans'])].groupby(['Commodity', 'Aggregated', 'Disaggregated', 'Province', 'Region']).sum(numeric_only=False).reset_index()
df_coffee['Crop'] = 'Coffee'
df_augment.loc[df_augment['Crop'].isin(['Coffee - Dried Berries', 'Coffee - Green Beans']), 'Disaggregated'] = True
df_coffee

,Commodity,Aggregated,Disaggregated,Province,Region,Crop,2010Q1,2010Q2,2010Q3,2010Q4,...,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4,2022Q1,2022Q2,2022Q3,2022Q4
0,Non-Food and Industrial Crops,True,False,Abra,CORDILLERA ADMINISTRATIVE REGION (CAR),Coffee,43.56,2.84,0.00,35.15,...,0.00,23.47,56.28,3.02,0.00,21.71,54.50,2.91,0.00,7.11
1,Non-Food and Industrial Crops,True,False,Agusan del Norte,REGION XIII (CARAGA),Coffee,92.49,71.00,66.00,84.56,...,42.09,72.12,42.14,42.57,51.09,81.11,33.47,40.87,20.13,29.04
2,Non-Food and Industrial Crops,True,False,Agusan del Sur,REGION XIII (CARAGA),Coffee,143.55,45.54,143.48,155.70,...,122.82,35.90,5.25,9.45,117.56,38.57,4.68,9.02,42.35,13.44
3,Non-Food and Industrial Crops,True,False,Aklan,REGION VI (WESTERN VISAYAS),Coffee,3.23,0.80,0.00,1.94,...,0.77,0.00,0.00,0.00,0.83,0.00,0.00,0.00,0.32,0.00
4,Non-Food and Industrial Crops,True,False,Albay,REGION V (BICOL REGION),Coffee,8.37,5.31,26.12,119.09,...,13.23,63.90,2.16,0.18,8.70,37.38,1.88,0.26,2.92,12.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Non-Food and Industrial Crops,True,False,Tawi-tawi,BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANA...,Coffee,14.01,1.65,125.49,41.15,...,91.50,37.95,15.30,6.60,94.62,40.71,17.10,8.67,33.24,14.30
79,Non-Food and Industrial Crops,True,False,Zambales,REGION III (CENTRAL LUZON),Coffee,0.00,0.00,0.00,0.00,...,0.00,2.88,0.00,0.51,0.41,1.95,0.00,0.47,0.14,0.51
80,Non-Food and Industrial Crops,True,False,Zamboanga Sibugay,REGION IX (ZAMBOANGA PENINSULA),Coffee,18.93,51.46,22.57,102.03,...,11.22,117.72,12.21,24.60,10.53,115.19,11.91,27.59,3.01,34.94
81,Non-Food and Industrial Crops,True,False,Zamboanga del Norte,REGION IX (ZAMBOANGA PENINSULA),Coffee,194.63,265.97,337.89,470.99,...,160.92,136.52,85.43,67.40,90.20,121.98,45.96,77.88,6.32,36.14


In [8]:
df_augment = pd.concat([df_augment, df_coffee])
df_augment = df_augment.set_index(['Crop', 'Commodity', 'Province', 'Region', 'Aggregated', 'Disaggregated'])
df_augment

2010Q1  \
Crop   Commodity                     Province            Region                                             Aggregated Disaggregated           
Abaca  Non-Food and Industrial Crops Abra                CORDILLERA ADMINISTRATIVE REGION (CAR)             False      False            0.00   
                                     Agusan del Norte    REGION XIII (CARAGA)                               False      False          110.41   
                                     Agusan del Sur      REGION XIII (CARAGA)                               False      False          289.58   
                                     Aklan               REGION VI (WESTERN VISAYAS)                        False      False          225.90   
                                     Albay               REGION V (BICOL REGION)                            False      False          453.00   
...                                                                                                                                      ...   
Coffee Non-Food and Industrial Crops Tawi-tawi           BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANAO... True       False           14.01   
                                     Zambales            REGION III (CENTRAL LUZON)                         True       False            0.00   
                                     Zamboanga Sibugay   REGION IX (ZAMBOANGA PENINSULA)                    True       False           18.93   
                                     Zamboanga del Norte REGION IX (ZAMBOANGA PENINSULA)                    True       False          194.63   
                                     Zamboanga del Sur   REGION IX (ZAMBOANGA PENINSULA)                    True       False           23.19   

                                                                                                                                      2010Q2  \
Crop   Commodity                     Province            Region                                             Aggregated Disaggregated           
Abaca  Non-Food and Industrial Crops Abra                CORDILLERA ADMINISTRATIVE REGION (CAR)             False      False            0.00   
                                     Agusan del Norte    REGION XIII (CARAGA)                               False      False          135.16   
                                     Agusan del Sur      REGION XIII (CARAGA)                               False      False          431.05   
                                     Aklan               REGION VI (WESTERN VISAYAS)                        False      False          247.25   
                                     Albay               REGION V (BICOL REGION)                            False      False          446.44   
...                                                                                                                                      ...   
Coffee Non-Food and Industrial Crops Tawi-tawi           BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANAO... True       False            1.65   
                                     Zambales            REGION III (CENTRAL LUZON)                         True       False            0.00   
                                     Zamboanga Sibugay   REGION IX (ZAMBOANGA PENINSULA)                    True       False           51.46   
                                     Zamboanga del Norte REGION IX (ZAMBOANGA PENINSULA)                    True       False          265.97   
                                     Zamboanga del Sur   REGION IX (ZAMBOANGA PENINSULA)                    True       False           23.78   

                                                                                                                                      2010Q3  \
Crop   Commodity                     Province            Region                                             Aggregated Disaggregated           
Abaca  Non-Food and Industrial Crops Abra                CORDILLERA ADMINISTRATIVE REGION (CAR)     

In [10]:
# 344 crops x 83 provinces
df_augment.to_csv('../data/crop-other-augmented.csv')